In [1]:
import re
import numpy as np
import random
import torch
import pandas as pd

from collections import Counter, defaultdict
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


# WSD

Ce notebook permet d'effectuer une comparaison des performances de deux algorithmes, un d'apprentissage supervisé et un d'apprentissage semi-supervisé, pour la tâche de Word Sense Disambiguation (WSD). 

Les deux méthodes sont les suivants :
- pour la classification supervisée, utilisation d'un MLP
- pour la classification semi-supervisée, utilisation d'un algorithme de constrained K-Means

Plusieurs tests sont effectués en considérant plusieurs mots à désambiguiser pour lesquels les performances de ces deux méthodes sont évaluées.



## Extraction des données

La classe Extractor permet d'accéder aux données et de les stocker dans des dictionnaires puis des embeddings. Elle permet de figer les données sur lesquelles les tests de classification seront ensuite faits. Elle comporte les méthodes suivantes :

- `extract_examples()` : pour extraire les données d'entraînement et de test provenant de fichiers XML FSE

- `extract_embeddings()` : pour extraire les embeddings à partir d'un fichier créé en amont, qui ne regroupe que les embeddings nécéssaires à notre jeu de données. Cette méthode est surtout utile pour drastiquement réduire le temps de chargement des embeddings

- `look_up()` : pour effectuer l'étape de look-up avant la classifcation

- `select_examples()` : pour sélectionner des données représentatives lorsqu'on ne considère pas toutes les données annotées. Pour qu'un ensemble de données soit représentatif, il faut que chaque étiquette présente dans les données totales soit présente au moins une fois dans le set d'entraînement sélectionné

- `define_instance()` : pour choisir les données qui seront classifiées, c'est-à-dire quelle proportion des données seront utilisées (80%, 50%, 25%...)


In [2]:
class Extractor :
    
    def __init__(self, data_path, gold_path, embeddings_path, context_size):
        '''
        A partir des données, extrait les examples (w2examples, w2senses) qui seront utilisés 
        lors de l'apprentissage.

        INPUT
        data_path (str) : chemin pour accéder aux données d'entraînement, au format XML
        gold_path (str) : chemin pour accéder aux numéros de sens correspondant aux données
                          d'entraînement
        embeddings_path (str) : chemin pour accéder aux embeddings, déjà extraits et 
                                présélectionnés pour ces données
        context_size (int) : taille de la fenêtre du contexte du mot à désambiguiser

        ARGUMENTS DE CLASSE
        self.w2examples (dict) : associe chaque lemme à une liste d'examples correspondant à son 
                                 utilisation dans une phrase du corpus. 
                                 examples (list) : liste de tous les couples 
                                  ([mots du contexte], numéro de sens) pour un lemme donné
        self.w2senses (dict) : associe chaque lemme à son ensemble de sens possible
        self.w2emb (dict) : associe chaque mot à son embedding
        self.selected_examples (liste) : liste d'examples qui contient au moins un example de 
                                         chaque sens du lemme choisi
        self.selected_examples_embs (liste) : même liste d'examples, mais avec les contextes 
                                              sous forme d'embedding
        self.X_train (set) : corpus d'apprentissage
        self.X_test (set) : numéros des sens correspondant au corpus d'apprentissage
        self.y_train (set) : corpus de test
        self.y_test (set) : numéros des sens correspondant au corpus d'apprentissage
        '''
        
        #récupération des données XML d'apprentissage fournies en argument
        tree = ET.parse(data_path)
        data_file = tree.getroot()

        #récupération des données txt correspondant aux gold class
        gold_file = open(gold_path, "r",encoding="utf-8")
        
        # self.w2examples (dict) : associe chaque lemme à une liste d'examples correspondant à son 
        #                          utilisation dans une phrase du corpus.
        # clé (str) : lemme du mot à désambiguiser
        # valeur (list) : liste de tous les couples ([mots du contexte], numéro de sens) pour ce 
        #                 lemme
        #
        # self.w2senses (dict) : associe chaque lemme à son ensemble de sens possible
        # clé (str) : lemme du mot à désambiguiser
        # valeur (set) : ensemble des sens possible pour ce lemme
        self.w2examples, self.w2senses = self.extract_examples_and_senses(data_file, gold_file, context_size)

        # self.w2emb (dict) : associe chaque mot à l'embedding représentant son contexte sommé
        # clé (str) : mot à désambiguiser
        # valeur (list) : embedding
        self.w2emb = self.extract_embeddings(embeddings_path)

        # liste d'examples qui contient au moins un example de chaque sens du lemme choisi
        # cette liste sera définie lorsque l'instance du mot à désambiguiser sera choisi
        self.selected_examples = []
        # même liste d'examples, mais avec les contextes sous forme d'embedding
        # cette liste sera définie lorsque l'instance du mot à désambiguiser sera choisi
        self.selected_examples_embs = []

        # les corpus seront définis lorsque l'instance du mot à désambiguiser sera choisi
        self.X_train = set()
        self.X_test = set()
        self.y_train = set()
        self.y_test = set()
    
    def extract_examples_and_senses(self, data_file, gold_file, context_size):
        '''
        Extrait les données à partir des fichiers de corpus d'apprentissage et de gold classes.

        INPUT
        data_file (Element): représentation des phrases du corpus d'apprentissage
        gold_file (TextIOWrapper): fichier contenant les numéros de sens (gold class) pour chaque
                                   mot à désambiguiser
        '''
    
        # clé (str) : lemme du mot à désambiguiser
        # valeur (list) : liste de tous les couples ([mots du contexte], numéro de sens) pour ce 
        #                 lemme
        w2examples = {}
        # clé (str) : lemme du mot à désambiguiser
        # valeur (set) : ensemble des sens possible pour ce lemme
        w2senses = defaultdict(set)

        # lecture du fichier gold
        gold_file = gold_file.readlines()
        
        # index de parcours dans le fichier gold
        i_gold = 0

        # pour chaque phrase du corpus
        for text_id in data_file:
            for sentence in text_id:

                # recherche de(s) l'indice(s) de(s) l'instance(s) pour savoir où se trouve la fenêtre
                i_instances = []
                for j in range(len(sentence)):
                    if sentence[j].tag == "instance":
                        i_instances.append(j)
                
                # tant qu'il y a des instances à repérer dans la phrase
                while len(i_instances) > 0:

                    # enregistrement du lemme de l'instance
                    instance = sentence[i_instances[0]].attrib["lemma"].lower()
                    
                    # si l'instance n'a pas encore de contexte
                    if instance not in w2examples : 
                        # le crée
                        w2examples[instance] = []

                    context = []
                    # pour chaque mot de la fenêtre
                    for k in range(-context_size, context_size + 1):
                        # si le mot est l'instance, enregistrement de son lemme
                        if k == 0:
                            context.append(instance)
                        # sinon enregistrement du mot du contexte
                        elif len(sentence) > i_instances[0] + k and i_instances[0] + k >= 0:
                            context.append(sentence[i_instances[0] + k].text.lower())
                    
                    # récupération des différents sens possibles dans le fichier gold 
                    gold_class = gold_file[i_gold].split("__")[1].split("_")[1]

                    # le fichiers gold et data ayant les données dans le même ordre, les instances
                    # et les sens peuvent être enregistrés en même temps dans leur dictionnaire
                    # respectif
                    w2senses[instance].add(gold_class)
                    w2examples[instance].append((context, gold_class))
                    
                    # l'instance lue est enlevée de la to-do list et l'index de parcours du fichier 
                    # gold est incrémenté pour passer à l'instance suivante
                    i_instances.pop(0)
                    i_gold += 1

        return w2examples, w2senses
    
    def extract_embeddings(self,path_embeddings) :
        '''
        Récupère les embeddings dans le fichier généré et associe à chaque mot son
        embedding.

        INPUT
        path_embeddings (str) : chemin pour accéder aux embeddings, déjà extraits et 
                                présélectionnés pour ces données

        OUTPUT
        w2emb (dict) : associe chaque mot à son embedding
        '''

        # lecture du fichier
        file_embeddings = open(path_embeddings , "r", encoding="UTF-8")

        # clé (str) : mot à désambiguiser
        # valeur (list) : embedding du mot
        w2emb = {}

        # pour chaque mot du fichier
        for line in file_embeddings.readlines():

            # séparation du mot et de l'embedding
            splitted_line = line.split(" ")
            word = splitted_line[0]
            embedding = list(map(float,splitted_line[1:]))
            # insertion du mot et de son embedding dans le dictionnaire
            w2emb[word] = embedding

        return w2emb

    def look_up(self, context) :
        '''
        Remplace dans les mots du vecteur de contexte par leur embedding et en fait la
        somme.

        INPUT
        context (list) : liste de taille (size_window*2)+1

        OUTPUT
        context_emb (list) : liste de taille size_embedding (300 ici)
        '''

        # taille d'un embedding : 300
        emb_size = len(list(self.w2emb.values())[0]) 
        # initialisation du contexte sous forme d'embedding
        context_emb = np.zeros(emb_size)

        # pour chaque mot du contexte
        for word in context :
            # s'il est présent dans le dictionnaire d'embeddings
            if word in self.w2emb :
                # il est ajouté à l'embedding représentant le contexte
                context_emb = np.add(context_emb, np.array(self.w2emb[word])) 

        return context_emb
    
    def select_examples(self, examples, senses, sample_size):
        '''
        Choisit des examples d'entraînement représentatifs du corpus selon un nombre
        imposé de données à choisir.

        INPUT
        examples (list) : liste d'examples pour le mot à désambiguiser
        n_senses (int) : nombre de senses associés à ce mot
        sample_size (float) : quantité des données d'entraînement considérés

        OUTPUT
        selected_examples (list) : liste d'examples qui contient au moins un example 
                                   de chaque sens
        '''

        selected_examples = []
        
        # pour chaque sens, on ajoute un example associé à ce sens, au hasard
        for sense in senses :
            selected_examples.append(random.choice(list(filter((lambda example:example[1]==sense),
                                                               examples))))
        # calcul du nombre d'examples qu'il reste à ajouter pour atteindre la quantité 
        # de données souhaitée
        size_to_add = round(sample_size*(len(examples)))-len(selected_examples)
        
        # ajoute ce nombre de données (non-présentes dans la liste) selectionnées au hasard
        selected_examples.extend(random.choices(list(filter((lambda example : example not in selected_examples),
                                                            examples)),
                                                    k=size_to_add))

        return selected_examples
    
    def select_train(x,y,senses,train_size=0.8):

            #pour chaque sens, on choisit au hasard un context et son étiquette
            selected_train = [random.choice(list(filter((lambda example:example[1]==sense),
                                                                zip(x,y)))) for sense in senses]
            
            #on choisit ensuite au hasard un même nombre de contextes et d'étiquettes différentes
            selected_test =random.choices(list(filter((lambda example : example not in selected_train),
                                                                zip(x,y))), k=len(selected_train))
            
            #on prend le reste d'example pour effectuer le split 
            remaining_X, remaining_y = zip(*(filter((lambda example:example not in selected_train and example not in selected_test),
                                                                zip(x,y))))
            
            X_train, X_test, y_train, y_test = train_test_split(remaining_X, remaining_y, train_size)
            
            #une fois le split effectué, on ajoute les données selectionnées en amont
            selected_X_train, selected_y_train = zip(*selected_train)
            selected_X_test, selected_y_test = zip(*selected_test)
            
            X_train.extend(selected_X_train)
            X_test.extend(selected_X_test)
            y_train.extend(selected_y_train)
            y_test.extend(selected_y_test)
            
            return X_train, X_test, y_train, y_test
    
    def define_instance(self, instance, data_size, affichage=True):
        '''
        Permet de définir l'instance du mot à désambiguiser et les examples à utiliser. 
        Fixe les derniers arguments de classe selon l'instance choisie.
        Contraint le choix de classification selon le nombre de données considéré.

        INPUT
        instance (str) : mot à désambiguiser
        data_size (float): quantité de données à considérer
        affichage (bool, optional): affiche une trace des options choisies
        '''

        # choisit des examples d'entraînement représentatifs du corpus selon le nombre
        # imposé de données
        self.selected_examples = self.select_examples(self.w2examples[instance],
                                                      self.w2senses[instance],
                                                      data_size)

        # liste de tous les contextes sous forme d'embedding
        x = [self.look_up(context) for context, gold in self.selected_examples]
        # liste de tous les numéros de sens
        y = [gold for context, gold in self.selected_examples]
        
        # liste des examples d'entrainement représentatifs sous forme d'embeddings
        self.selected_examples_embs = list(zip(x,y))
        
        # création des corpus d'entrainement et de test
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(x, y, train_size=0.8)
        #self.X_train, self.X_test, self.y_train, self.y_test = self.select_train(x, y, self.w2senses[instance])
        
        # si une trace est souhaitée
        if affichage :
            print("instance :", instance)
            print(f'{data_size*100}% des données annotées considérées')
            print("nombre de données d'entraînement : ", len(self.X_train))
            print("étiquettes possibles pour cette instance : ", self.w2senses[instance])
            print("étiquettes présentes dans les données d'entraînement :", Counter(self.y_train)) # TODO : ce n'est pas représentatif car il manque souvent au moins une étiquette
        return
    
    def get_sets(self):
        return self.X_train, self.X_test, self.y_train, self.y_test
    
    def get_examples(self):
        return self.selected_examples
    
    def get_embs(self):
        return self.selected_examples_embs

In [3]:
# chemin du corpus annoté
data_path = "../donnees/FSE-1.1-191210/FSE-1.1.data.xml"
# chemin pour récupérer les gold class du corpus annoté
gold_path = "../donnees/FSE-1.1-191210/FSE-1.1.gold.key.txt"
# choix de la fenêtre du contexte
context_size = 4
# chemin pour récupérer les embeddings afin de faire l'opération look-up
# Les embeddings sont extraits de fasttext
embeddings_path = "embeddings.txt"

In [4]:
ext = Extractor(data_path, gold_path, embeddings_path, context_size)
ext.define_instance("aboutir",1)
X_train, X_test, y_train, y_test = ext.get_sets()
print("\nexemple d'example avec contexte et gold class :",ext.get_examples()[0])
print("\nau format embedding : ",ext.get_embs()[0])

instance : aboutir
100% des données annotées considérées
nombre de données d'entraînement :  40
étiquettes possibles pour cette instance :  {'3', '2', '4', '1'}
étiquettes présentes dans les données d'entraînement : Counter({'3': 31, '4': 7, '1': 1, '2': 1})

exemple d'example avec contexte et gold class : (['de', 'déportés', ',', 'ont', 'aboutir', 'à', 'ces', 'nouvelles', 'inculpations'], '3')

au format embedding :  (array([-1.6300e-01,  2.6840e-01,  2.2830e-01, -1.7200e-01, -8.0960e-01,
        4.0800e-02,  1.8990e-01, -1.2510e-01,  1.2300e-02, -3.4700e-01,
       -3.0400e-02, -7.0570e-01,  8.0400e-02,  2.5460e-01, -1.3730e-01,
        3.7610e-01,  3.5900e-02,  2.8560e-01, -2.0030e-01,  1.4770e-01,
        2.8170e-01, -1.4960e-01, -1.4780e-01,  5.1560e-01,  4.9800e-02,
        4.9910e-01,  1.9530e-01, -4.2100e-02,  2.2470e-01, -1.5500e-02,
       -7.0400e-02,  7.4700e-02, -2.6710e-01,  5.1300e-02,  5.9700e-02,
        3.4140e-01, -6.4250e-01, -5.4700e-02,  1.4000e-01, -3.2500e-01,
 


## Classification supervisé

On construit une classe Classifieur qui va nous permettre de prédire.


In [5]:
class Classifieur :
    
    def __init__(self,X_train, X_test, y_train, y_test) :
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test 
        
    def classify(self,affichage=False) :
        """Permet de prédire et d'afficher les données de classification.

        Args:
            affichage (bool, optional): Defaults to False.

        Returns:
            list: prédiction
        """
        clf = MLPClassifier(random_state=1,hidden_layer_sizes=(100,)) 
        clf.fit(self.X_train, self.y_train)
        y_pred = clf.predict(self.X_test)
        
        if affichage :
            print("prediction :",y_pred)
            print("gold :",self.y_test)
            print("accuracy score : ", accuracy_score(y_pred,self.y_test),"\n")
            
        return y_pred

In [6]:
Clf = Classifieur(X_train,X_test,y_train,y_test)
Clf.classify(affichage=True)

prediction : ['4' '3' '3' '3' '3' '4' '3' '3' '4' '3']
gold : ['4', '3', '3', '3', '3', '3', '3', '3', '4', '3']
accuracy score :  0.9 



array(['4', '3', '3', '3', '3', '4', '3', '3', '4', '3'], dtype='<U1')

Dans un second temps, on effectue nos tests sur plusieurs classifieurs en choisissant un pas de descente dans la quantité des données annotées considérées. Pour chaque classifieur et chaque quantitée de données considérées, on effectue n_repeat classifications pour obtenir une accuracy moyenne représentative du classifieur. Par conséquent, pour n_repeat=5 et step=0.25, nous obtiendrons pour chaque classifieur une liste d'accuracies correspondante à la moyenne des accuracies de 5 prédictions pour 100%, 75%, 50% et 25% des données.

In [7]:
def get_mean_accuracy(instance,data_size,n_repeat,affichage=True):
        """Permet de retourner une moyenne d'accuracies pour un nombre de classifications donné.

        Args:
            n_repeat (int): nombre de classifications à tester
            affichage (bool, optional): Defaults to False.

        Returns:
            int: moyenne des accuracies computed
        """
        accuracies = []
        
        for i in range(n_repeat) :
            
            #pour chaque test, on crée un nouveau classifieur, qui s'appuie toujours sur la même quantité de données mais avec un autre tirage
            #par exemple, pour 75% des données, on obtiendra des accuracies différentes en fonction des tirages de ces données
            ext.define_instance(instance,data_size,False)
            X_train, X_test, y_train, y_test = ext.get_sets()
            Clf = Classifieur(X_train, X_test, y_train, y_test)
            y_pred = Clf.classify()
            
            accuracies.append(accuracy_score(y_pred,y_test))
            mean = sum(accuracies)/len(accuracies)
            
        if affichage :
            print(f'{data_size*100}% des données annotées considérées')
            print("instance :",instance)
            print("accuracies :",accuracies)
            print("accuracy moyenne :",mean)
            print()
        return mean

def get_accuracies(instances,step,n_repeat,affichage=False):
            """Permet d'obtenir une accuracy moyenne par lemme pour une certaine quantité de données considérée.
            Produit un fichier csv.

            Args:
                instances (string): mot à désambiguiser
                step (float): pas de descente dans la quantité de données à considérer
                n_repeat (int): nombre de classifications tests à effectuer

            Returns:
                dictionnary: associe à chaque instance sa liste d'accuracies moyenne 
            """
            instance2acc = {instance : [] for instance in instances}
            moyennes = []
            data_sizes =[]
            
            for i in range(round(1.0/step)): #pour chaque quantité de données observée
                
                data_size = 1.0 - (step*float(i))
                data_sizes.append(data_size)
                
                for instance in instances : #pour chaque mot ambigü sélectionné
                    instance2acc[instance].append(get_mean_accuracy(instance,data_size,n_repeat,affichage)) #on récupère son accuracy

                #on boucle sur les valeurs obtenues pour obtenir leur moyenne
                print(list(instance2acc.values()))
                print([accuracies[i] for accuracies in instance2acc.values()])
                moyennes.append(sum([accuracies[i] for accuracies in instance2acc.values()])/len(instance2acc))
  
            instance2acc["moyennes"] = moyennes
            
            tab = instance2acc.copy()
            tab["data_sizes"] = data_sizes 
            df = pd.DataFrame(tab)
            df.set_index("data_sizes")
            df.to_csv(f"accuracies_par_lemme_{n_repeat}_repet.csv")
            return instance2acc     

In [8]:
#A définir
#Nombre de classifieurs choisis au hasard à tester
#n_rand_instances = 3
#Pas de descente dans la quantité de données considérées
step = 0.25
#Nombre de classifications pour un classifieur pour obtenir une accuracy moyenne
n_repeat = 5

In [9]:
#instances = random.choices(list(ext.w2examples.keys()),k=n_rand_instances)
instances = list(ext.w2examples.keys())
get_accuracies(instances,step,n_repeat)

c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimiza

[[0.9], [0.78], [0.9199999999999999], [0.9400000000000001], [0.925], [0.86], [0.9800000000000001], [0.82], [0.8363636363636363], [0.7399999999999999], [0.8363636363636363], [0.8799999999999999], [0.76], [0.8400000000000001], [0.9], [0.9800000000000001], [0.7454545454545455], [0.9399999999999998], [0.74], [0.8400000000000001], [0.96], [0.6799999999999999], [0.8], [0.8600000000000001], [0.7999999999999999], [0.74], [0.6399999999999999], [0.6599999999999999], [0.82], [0.890909090909091], [0.8200000000000001], [0.58], [0.9428571428571428], [0.5800000000000001], [0.8800000000000001], [0.8200000000000001], [0.6], [1.0], [0.78], [0.9800000000000001], [0.9199999999999999], [0.8000000000000002], [0.7799999999999999], [0.96], [0.64], [0.9800000000000001], [0.64], [0.6599999999999999], [0.9800000000000001], [0.8363636363636365], [0.7999999999999999], [0.76], [0.9090909090909092], [0.85], [0.9400000000000001], [0.76], [0.9400000000000001], [0.8], [0.8800000000000001], [0.9636363636363636], [1.0], 

c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimiza

[[0.9, 0.85], [0.78, 0.575], [0.9199999999999999, 0.775], [0.9400000000000001, 1.0], [0.925, 0.7], [0.86, 0.85], [0.9800000000000001, 0.975], [0.82, 0.65], [0.8363636363636363, 0.7], [0.7399999999999999, 0.65], [0.8363636363636363, 0.825], [0.8799999999999999, 0.875], [0.76, 0.75], [0.8400000000000001, 0.75], [0.9, 0.775], [0.9800000000000001, 0.725], [0.7454545454545455, 0.55], [0.9399999999999998, 0.85], [0.74, 0.625], [0.8400000000000001, 0.8571428571428571], [0.96, 0.875], [0.6799999999999999, 0.675], [0.8, 0.65], [0.8600000000000001, 0.85], [0.7999999999999999, 0.7428571428571429], [0.74, 0.55], [0.6399999999999999, 0.725], [0.6599999999999999, 0.675], [0.82, 0.675], [0.890909090909091, 0.75], [0.8200000000000001, 0.85], [0.58, 0.6571428571428571], [0.9428571428571428, 0.9666666666666668], [0.5800000000000001, 0.725], [0.8800000000000001, 0.975], [0.8200000000000001, 0.775], [0.6, 0.55], [1.0, 1.0], [0.78, 0.7], [0.9800000000000001, 1.0], [0.9199999999999999, 0.8], [0.800000000000

c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimiza

[[0.9, 0.85, 0.76], [0.78, 0.575, 0.64], [0.9199999999999999, 0.775, 0.7999999999999999], [0.9400000000000001, 1.0, 0.8400000000000001], [0.925, 0.7, 0.6], [0.86, 0.85, 0.8], [0.9800000000000001, 0.975, 1.0], [0.82, 0.65, 0.5599999999999999], [0.8363636363636363, 0.7, 0.6333333333333333], [0.7399999999999999, 0.65, 0.6], [0.8363636363636363, 0.825, 0.6], [0.8799999999999999, 0.875, 0.72], [0.76, 0.75, 0.64], [0.8400000000000001, 0.75, 0.44000000000000006], [0.9, 0.775, 0.6799999999999999], [0.9800000000000001, 0.725, 0.8], [0.7454545454545455, 0.55, 0.4666666666666666], [0.9399999999999998, 0.85, 0.76], [0.74, 0.625, 0.56], [0.8400000000000001, 0.8571428571428571, 0.56], [0.96, 0.875, 0.8800000000000001], [0.6799999999999999, 0.675, 0.48], [0.8, 0.65, 0.6799999999999999], [0.8600000000000001, 0.85, 0.5599999999999999], [0.7999999999999999, 0.7428571428571429, 0.64], [0.74, 0.55, 0.6], [0.6399999999999999, 0.725, 0.6400000000000001], [0.6599999999999999, 0.675, 0.6399999999999999], [0.8

c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\anais\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[[0.9, 0.85, 0.76, 0.6666666666666666], [0.78, 0.575, 0.64, 0.26666666666666666], [0.9199999999999999, 0.775, 0.7999999999999999, 0.4666666666666667], [0.9400000000000001, 1.0, 0.8400000000000001, 0.7999999999999999], [0.925, 0.7, 0.6, 0.4], [0.86, 0.85, 0.8, 0.5999999999999999], [0.9800000000000001, 0.975, 1.0, 0.6], [0.82, 0.65, 0.5599999999999999, 0.26666666666666666], [0.8363636363636363, 0.7, 0.6333333333333333, 0.6666666666666666], [0.7399999999999999, 0.65, 0.6, 0.2], [0.8363636363636363, 0.825, 0.6, 0.5333333333333333], [0.8799999999999999, 0.875, 0.72, 0.39999999999999997], [0.76, 0.75, 0.64, 0.5333333333333333], [0.8400000000000001, 0.75, 0.44000000000000006, 0.4], [0.9, 0.775, 0.6799999999999999, 0.39999999999999997], [0.9800000000000001, 0.725, 0.8, 0.7333333333333333], [0.7454545454545455, 0.55, 0.4666666666666666, 0.26666666666666666], [0.9399999999999998, 0.85, 0.76, 0.8666666666666666], [0.74, 0.625, 0.56, 0.3333333333333333], [0.8400000000000001, 0.8571428571428571, 0.

{'aboutir': [0.9, 0.85, 0.76, 0.6666666666666666],
 'investir': [0.78, 0.575, 0.64, 0.26666666666666666],
 'traduire': [0.9199999999999999,
  0.775,
  0.7999999999999999,
  0.4666666666666667],
 'témoigner': [0.9400000000000001,
  1.0,
  0.8400000000000001,
  0.7999999999999999],
 'juger': [0.925, 0.7, 0.6, 0.4],
 'justifier': [0.86, 0.85, 0.8, 0.5999999999999999],
 'viser': [0.9800000000000001, 0.975, 1.0, 0.6],
 'prononcer': [0.82, 0.65, 0.5599999999999999, 0.26666666666666666],
 'accomplir': [0.8363636363636363,
  0.7,
  0.6333333333333333,
  0.6666666666666666],
 'convenir': [0.7399999999999999, 0.65, 0.6, 0.2],
 'acquérir': [0.8363636363636363, 0.825, 0.6, 0.5333333333333333],
 'achever': [0.8799999999999999, 0.875, 0.72, 0.39999999999999997],
 'observer': [0.76, 0.75, 0.64, 0.5333333333333333],
 'adapter': [0.8400000000000001, 0.75, 0.44000000000000006, 0.4],
 'admettre': [0.9, 0.775, 0.6799999999999999, 0.39999999999999997],
 'entraîner': [0.9800000000000001, 0.725, 0.8, 0.73333

## Classification semi-supervisée

In [10]:
class K_Means():
    ''' 
    classifieur K-means pour un mot particulier
    '''

    def __init__(self, examples):
        '''
        Instancie les différentes variables utiles pour l'algorithme du K-means

        examples : liste d'examples dont le mot à désambiguiser est le même pour 
                   chaque example
        example : couple d'un mot avec son contexte de fenêtre 4 (sous forme 
                  d'embedding) et du numéro de sens attendu du mot à désambiguiser 
                  (gold class sous forme d'integer)
        '''

        # transforme l'ensemble des examples en une liste pour pouvoir garder le 
        # même indice pour chaque example par la suite
        self.examples = list(examples)
        # transforme les embeddings en tensors
        self.tensors_examples = [example[0] for example in self.examples]
        # détermine le nombre de sens possibles k (donc le nombre de clusters) 
        # à l'aide des données
        self.k = self.nb_senses()
        # initialisation de centroids : pour chaque sens, un example est pris au hasard
        self.tensors_centroids = [random.choice(example) 
                                  for example in self.examples_of_same_sense().values()]
        # initialisation de clusters : tous les examples sont associés au cluster 0
        self.clusters = np.zeros(len(examples))

    def nb_senses(self):
        '''
        Renvoie le nombre de sens existants dans un ensemble d'examples
        '''

        known_senses = []
        # pour chaque exemple
        for example in self.examples:
            # si le sens attendu (gold class) n'a pas encore été rencontré
            if example[1] not in known_senses:
                # l'ajoute à la liste des sens possibles
                known_senses.append(example[1])
        # renvoie le nombre de sens
        return len(known_senses)
    
    def examples_of_same_sense(self):
        '''
        Regroupe les contextes des examples dans un dictionnaire triés selon le 
        sens du mot à désambiguiser
        '''

        # clé : numéro du sens
        # valeur : liste de contextes avec ce sens en gold class
        sense2examples = {}
        # pour chaque example
        for example in self.examples:
            # si sa gold class n'a pas été déjà rencontrée
            if example[1] not in sense2examples:
                # ajoute une clé pour cette gold class
                sense2examples[example[1]] = []
            # ajoute le contexte au dictionnaire correspondant au sens utilisé
            sense2examples[example[1]].append(example[0])

        return sense2examples
    
    def learn_clusters(self):
        '''
        Algorithme de K-Means
        Retourne les coordonnées de chaque centroide ainsi que le cluster auquel 
        appartient chaque example
        '''

        # différence initialisée à Vrai
        diff = True
        
        # tant qu'il y a une différence entre l'ancienne liste et la nouvelle 
        # liste de centroides
        while diff:

            # CALCUL DES DISTANCES ENTRE CHAQUE EXAMPLE ET CHAQUE CENTROIDE

            # pour chaque couple (indice, coordonnées) dans les examples
            for i, tensor_example in enumerate(self.tensors_examples):
                # initialisation de la distance minimum à l'infini
                min_dist = float('inf')
                # pour chaque couple (indice, coordonnées) dans les centroides
                for j, tensor_centroid in enumerate(self.tensors_centroids):
                    # calcul de la distance entre cet example et ce centroide
                    d = 0
                    for k in range(len(tensor_example)):
                        d += (tensor_centroid[k].item() - tensor_example[k].item())**2
                    d = np.sqrt(d)
                    # si une distance plus faible est trouvée
                    if min_dist > d:
                        # la distance ainsi que le centroide sont stockés
                        min_dist = d
                        self.clusters[i] = j
            
            # CALCUL DES NOUVEAUX CENTROIDES

            # calcul des nouveaux centroides en utilisant le point au milieu de tous les
            # autres points du même cluster
            new_centroids = pd.DataFrame(self.tensors_examples).groupby(by = self.clusters).mean()
            # transforme ces nouveaux centroides en tensors
            tensors_new_centroids = []
            for i in range(len(new_centroids.index)):
                colums = []
                for j in range(len(new_centroids.columns)):
                    colums.append(int(new_centroids.iat[i,j]))
                tensors_new_centroids.append(torch.tensor(colums))

            # MISE A JOUR DES CENTROIDES

            count_diff = 0
            # pour chaque centroide
            for i in range(len(self.tensors_centroids)):
                # si l'ancien centroide et le nouveau ne sont pas les mêmes
                if not(torch.equal(self.tensors_centroids[i], tensors_new_centroids[i])):
                    count_diff += 1
                    # met à jour le centroide
                    self.tensors_centroids = tensors_new_centroids
            # s'il n'y a eu aucune différence entre les anciens et les nouveaux centroides, 
            # la boucle while se termine
            if count_diff == 0:
                diff = False

In [11]:
examples = ext.get_embs()
k_means = K_Means(examples)
k_means.learn_clusters()

TypeError: equal(): argument 'input' (position 1) must be Tensor, not numpy.ndarray